In [ ]:

import  upstox_client #import Upstox
# from upstox_client.models import AccessToken
from upstox_client.api.history_api import HistoryApi

import pandas as pd
 

from upstox_client.rest import ApiException
import upstox_client
# Replace with your API key and access token
# Broker Configuration
EDIRECT_URI = 'http://127.0.0.1:5000/upstox/callback' 

# Configure the Upstox client
configuration = upstox_client.Configuration()
configuration.access_token = ACCESS_TOKEN
client = upstox_client.ApiClient(configuration)

# Instantiate the HistoryApi object
api_instance = HistoryApi(client)

# Example parameters (replace with your desired values)
instrument_key = "NSE_EQ|INE002A01018"  # Example: Reliance Industries NSE_EQ|INE002A01018
interval = "1minute"
to_date = "2025-05-01"  # Today's date
from_date = "2025-04-10"  # Previous day

try:
    # Call the get_historical_candle_data1 API endpoint
    api_response = api_instance.get_historical_candle_data1(
        instrument_key=instrument_key,
        interval=interval,
        to_date=to_date,
        from_date=from_date,
        api_version='v2'
    )

    # Print the response
    # print("Response:", api_response)

    # Extract data from the response
    data = api_response.data.candles

    # Create a Pandas DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi'])


except ApiException as e:
    # Handle any errors
    print("Exception when calling HistoryApi->get_historical_candle_data1: %s\n" % e)

import pandas as pd

# Convert to DataFrame
#df = pd.DataFrame(data['data']['candles'], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set timestamp as index
df.set_index('timestamp', inplace=True)

# Resample to 5-minute intervals
df_resampled = df.resample('5T').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()


LOOKBACK_PERIOD = 48
VOLUME_STD_MULTIPLIER = 4

# Calculate rolling mean and std
df_resampled['vol_mean'] = df_resampled['volume'].rolling(window=LOOKBACK_PERIOD).mean()
df_resampled['vol_std'] = df_resampled['volume'].rolling(window=LOOKBACK_PERIOD).std()

# Identify significant candles
df_resampled['vol_threshold'] = df_resampled['vol_mean'] + VOLUME_STD_MULTIPLIER * df_resampled['vol_std']
df_resampled['is_significant'] = df_resampled['volume'] > df_resampled['vol_threshold']

# Identify S/R levels
support = df_resampled[df_resampled['is_significant'] & (df_resampled['close'] > df_resampled['open'])]
resistance = df_resampled[df_resampled['is_significant'] & (df_resampled['close'] < df_resampled['open'])]


SR_PROXIMITY_THRESHOLD = 0.002  # 0.2%

# Current candle close price
current_price = df_resampled.iloc[-1]['close']

# Check proximity to support
support_near = support[abs(support['close'] - current_price) / current_price < SR_PROXIMITY_THRESHOLD]

# Check proximity to resistance
resistance_near = resistance[abs(resistance['close'] - current_price) / current_price < SR_PROXIMITY_THRESHOLD]



In [ ]:
support.head()

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot closing prices
plt.plot(df_resampled.index, df_resampled['close'], label='Close Price')

# Plot support and resistance levels
plt.scatter(support.index, support['close'], color='green', label='Support', marker='^')
plt.scatter(resistance.index, resistance['close'], color='red', label='Resistance', marker='v')

plt.title('5-Minute Candles with Support and Resistance Levels')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot closing prices
fig.add_trace(go.Scatter(
    x=df_resampled.index,
    y=df_resampled['close'],
    mode='lines',
    name='Close Price'
))

# Plot support levels
fig.add_trace(go.Scatter(
    x=support.index,
    y=support['close'],
    mode='markers',
    name='Support',
    marker=dict(color='green', symbol='triangle-up', size=10)
))

# Plot resistance levels
fig.add_trace(go.Scatter(
    x=resistance.index,
    y=resistance['close'],
    mode='markers',
    name='Resistance',
    marker=dict(color='red', symbol='triangle-down', size=10)
))

# Layout configuration
fig.update_layout(
    title='5-Minute Candles with Support and Resistance Levels',
    xaxis_title='Time',
    yaxis_title='Price',
    legend=dict(x=0, y=1),
    template='plotly_white'
)

fig.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import mplfinance as mpf


# Plot support levels
plt.scatter(support.index, support['close'], color='black', label='Support', marker='^')

# Plot resistance levels
plt.scatter(resistance.index, resistance['close'], color='black', label='Resistance', marker='v')

# Add a legend to the plot
plt.legend()
# Create the candlestick chart
# mpf.plot(df_resampled, type='candle', volume=True, style='yahoo', figsize=(10, 6))
plt.show()

In [ ]:
import plotly.graph_objects as go

# Create the candlestick chart
fig = go.Figure(data=[
    go.Candlestick(
        x=df_resampled.index,
        open=df_resampled['open'],
        high=df_resampled['high'],
        low=df_resampled['low'],
        close=df_resampled['close'],
        name='Candles'
    )
])

# Plot support levels
fig.add_trace(go.Scatter(
    x=support.index,
    y=support['close'],
    mode='markers',
    name='Support',
    marker=dict(color='green', symbol='triangle-up', size=8)
))

# Plot resistance levels
fig.add_trace(go.Scatter(
    x=resistance.index,
    y=resistance['close'],
    mode='markers',
    name='Resistance',
    marker=dict(color='red', symbol='triangle-down', size=8)
))

# Layout with smaller font sizes
fig.update_layout(
    title='5-Minute Candles with Support and Resistance Levels',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis=dict(
        type='category',
        tickfont=dict(size=10),  # smaller x-axis labels
        showgrid=True
    ),
    yaxis=dict(
        tickfont=dict(size=10),  # smaller y-axis labels
    ),
    font=dict(size=12),  # overall font size (title, legend, etc.)
    legend=dict(font=dict(size=10)),
    template='plotly_white',
    height=600,  # optional: you can also adjust figure height
)

fig.show()


In [3]:

from tradingview_screener import Query, col
 
# Set display options to prevent column wrapping
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0) # Adjust as needed, 0 will use the full available width


writeHeader=True
while True :
    tvQuery = Query().set_markets('india')#.set_index('SYML:NSE;CNX500') #INDEX
    # tvQuery.set_markets('india') #MARKET
    #'indexes','name', 'close', 'volume',
    _,dfNew = (tvQuery
    .select('name', 'premarket_change','premarket_open', 'premarket_high','premarket_low','premarket_close','premarket_gap','premarket_volume')
    # .select('name', 'close', 'volume', 'relative_volume_10d_calc','volume_change|1','relative_volume_intraday|5','average_volume','VWAP|1','VWAP|5','VWMA|5','VWMA|15')
    # #'relative_volume_10d_calc','volume_change|1','relative_volume_intraday|5','relative_volume','VWMA','premarket_change','premarket_change_abs','premarket_change_from_open','premarket_change_from_open_abs','premarket_close','premarket_gap','premarket_high','premarket_low','premarket_open','premarket_volume')
    .where(      
            # # col('name')=='NSE:HINDMOTORS',
            col('premarket_gap') > 1, 
        )
    # .order_by('relative_volume_intraday|5', ascending=False)
    .order_by('volume|1', ascending=False)
    
    .get_scanner_data())

    print(dfNew)
    break

            ticker        name  premarket_change  premarket_open  premarket_high  premarket_low  premarket_close  premarket_gap  premarket_volume
0         BSE:CITL        CITL          3.973510            1.57            1.57           1.57             1.57       3.973510            124712
1      BSE:STANCAP     STANCAP          2.127660            0.48            0.48           0.48             0.48       2.127660             71482
2       BSE:QUASAR      QUASAR          2.702703            0.76            0.76           0.76             0.76       2.702703             13163
3          NSE:VMM         VMM          3.890075          111.90          111.90         111.90           111.90       3.890075                10
4      BSE:PRADHIN     PRADHIN          2.325581            0.44            0.44           0.44             0.44       2.325581           1721105
5         BSE:GNRL        GNRL          2.163179           43.45           43.45          43.45            43.45       2.163

In [ ]:
%pip install ipywidgets

In [ ]:
%matplotlib widget


In [ ]:
# import json
# import requests

# def get_instruments():
#     """
#     Retrieves the Upstox instrument key for a given symbol.

#     Args:
#         symbol (str): The symbol (e.g., "RELIANCE") of the instrument.

#     Returns:
#         str: The Upstox instrument key, or None if not found.
#     """

#     # Replace with your actual Upstox API key and the base URL
#     api_key = API_KEY
#     base_url = "https://api.upstox.com/v2/instruments"

#     # Headers for the API request
#     headers = {
#         "Authorization": f"Bearer {api_key}",
#         "Accept": "application/json",
#     }

#     # Make the API request to retrieve the instruments
#     try:
#         response = requests.get(base_url, headers=headers)
#         response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
#     except requests.exceptions.RequestException as e:
#         print(f"Error during API request: {e}")
#         return None

#     # Parse the JSON response
#     try:
#         instruments = json.loads(response.content)
#     except json.JSONDecodeError:
#         print("Error decoding JSON response.")
#         return None
#     print(instruments)
#     return instruments


# def get_instrument_key_from_symbol(symbol,instruments):
#     # Search for the instrument key based on the symbol
#     for instrument in instruments:
#         if instrument["symbol"] == symbol:
#             return instrument["instrument_key"]

#     # If the symbol is not found, return None
#     print(f"Symbol '{symbol}' not found in instrument list.")
#     return None


# # Example Usage:
# symbol_to_find = "RELIANCE"  # Replace with the symbol you're searching for
# instruments = get_instruments()

# instrument_key = get_instrument_key_from_symbol(symbol_to_find,instruments)
# if instrument_key:
#     print(f"Instrument key for '{symbol_to_find}': {instrument_key}")
# else:
#     print(f"Instrument key for '{symbol_to_find}' not found.")


In [ ]:
instruments